In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import cv2
import os
import numpy as np
import h5py
import imutils

In [4]:
## load data set
## run this only if you already have the saved data in your drive

hf = h5py.File('x_train.h5', 'r')
X_train = np.array(hf['QuickDraw'][:])
hf = h5py.File('y_train.h5', 'r')
Y_train= np.array(hf['QuickDraw'][:])
hf = h5py.File('x_valid.h5', 'r')
X_valid = np.array(hf['QuickDraw'][:])
hf = h5py.File('y_valid.h5', 'r')
Y_valid = np.array(hf['QuickDraw'][:])
hf = h5py.File('x_test.h5', 'r')
X_test = np.array(hf['QuickDraw'][:])
hf = h5py.File('y_test.h5', 'r')
Y_test = np.array(hf['QuickDraw'][:])

In [5]:
## load the model
## run this only if you already have the saved model in your drive

loaded_model = tf.keras.models.load_model('cnn.h5')
loss, accuracy = loaded_model.evaluate(X_test, Y_test, verbose=1)
print('Test loss: ', loss)
print('Test accuracy: ', accuracy)

69/69 [==============================] - 1s 18ms/step - loss: 0.0940 - accuracy: 0.9917
Test loss:  0.09395226091146469
Test accuracy:  0.9917317628860474


In [6]:
## https://stackoverflow.com/questions/37177811/crop-rectangle-returned-by-minarearect-opencv-python

def crop_minAreaRect(rect, box, frame):
    w, h, d = frame.shape
    mult = 1 # I wanted to show an area slightly larger than my min rectangle set this to one if you don't
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped1 = cv2.getRectSubPix(frame[:,:,0], size, center)    
    cropped1 = cv2.warpAffine(cropped1, M, size)

    cropped2 = cv2.getRectSubPix(frame[:,:,1], size, center)    
    cropped2 = cv2.warpAffine(cropped2, M, size)

    cropped3 = cv2.getRectSubPix(frame[:,:,2], size, center)    
    cropped3 = cv2.warpAffine(cropped3, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated1 = cv2.getRectSubPix(cropped1, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    croppedRotated2 = cv2.getRectSubPix(cropped2, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    croppedRotated3 = cv2.getRectSubPix(cropped3, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))

#     cv2.circle(frame, center, 10, (0,255,0), -1) # again this was mostly for debugging purposes
#     cv2.rectangle(frame, 
#                  (int(center[0] - croppedW/2), int(center[1] - croppedH/2)), 
#                  (int(center[0] + croppedW/2), int(center[1] + croppedH/2)), 
#                  (255, 255, 255), 
#                  2) # again this was mostly for debugging purposes
    
    return np.concatenate((croppedRotated1.reshape(croppedRotated1.shape[0], croppedRotated1.shape[1], 1), croppedRotated2.reshape(croppedRotated2.shape[0], croppedRotated2.shape[1], 1), croppedRotated3.reshape(croppedRotated3.shape[0], croppedRotated3.shape[1], 1)), axis=2)

In [9]:
## test with video

backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=False, varThreshold=20)
capture = cv2.VideoCapture('12.mp4')
fps = 100
MIN_AREA = 600

if not capture.isOpened:
    print('Unable to open the video')
    exit(0)

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    # convert to gray + resize + apply filter
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    fgMask = backSub.apply(gray)
    fgMask = cv2.medianBlur(fgMask,5)
    
    # get contours
    cnts = cv2.findContours(
        fgMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    for c in cnts:
        rect = cv2.minAreaRect(c)
        if (rect[1][0]*rect[1][1]) < MIN_AREA:
            continue
        
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(frame,[box],0,(0,255,0),2)
        
        img_crop = crop_minAreaRect(rect, box, frame)    

        img_crop_1 = cv2.resize(img_crop[:, :, 0], (64, 64), interpolation = cv2.INTER_AREA)
        img_crop_2 = cv2.resize(img_crop[:, :, 1], (64, 64), interpolation = cv2.INTER_AREA)
        img_crop_3 = cv2.resize(img_crop[:, :, 2], (64, 64), interpolation = cv2.INTER_AREA)

        img_crop = np.concatenate((img_crop_1.reshape(64, 64, 1), img_crop_2.reshape(64, 64, 1), img_crop_3.reshape(64, 64, 1)), axis=2)
        
        Y_box_pred = np.sum(loaded_model.predict(img_crop.reshape(1, 64, 64, 3)))
        
        if Y_box_pred == 1:
            cv2.putText(frame, "Person", (box[2][0], box[2][1]),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        if Y_box_pred == 0:
            cv2.putText(frame, "Non", (box[2][0], box[2][1]),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        
        
        
        
#         if cv2.contourArea(c) < MIN_AREA:
#             continue

#         (x, y, w, h) = cv2.boundingRect(c)
#         cut_gray = gray_copy[y:y+h,x:x+w]
#         cut_gray = cv2.resize(cut_gray, (64,64), interpolation = cv2.INTER_AREA)
#         Y_box_pred = np.sum(nn_model_2.predict(cut_gray.reshape(1, -1)))
#         if Y_box_pred == 1:
#             cv2.putText(frame, "Person", (x, y-10),
#                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
#         if Y_box_pred == 0:
#             cv2.putText(frame, "Non", (x, y-10),
#                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  
        
    cv2.imshow('Frame', frame)
#     cv2.imshow('FG Mask', fgMask)
#     cv2.imshow('Gray copy', gray_copy)
    if cv2.waitKey(int((1/fps)*1000)) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()